# Functions to analyze Images

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from ultralytics import YOLO
import numpy as np
import cv2
import gc
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Función para encontrar foto de hogar Original en la carpeta

In [ ]:
# Function to find an image file that matches the search term
def find_matching_image(search_term, images_folder):
    # Common image file extensions
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff']
    
    # Convert search term to lowercase for case-insensitive matching
    search_term_lower = search_term.lower()
    
    # List all files in the directory
    for file in os.listdir(images_folder):
        file_lower = file.lower()
        
        # Check if the file is an image and matches the search term
        if any(file_lower.endswith(ext) for ext in image_extensions):
            
            # Match with the filename (without extension)
            if os.path.splitext(file_lower)[0] == search_term_lower:
                return os.path.join(images_folder, file)

    
    # No matching image found
    return None


## YOLO

In [ ]:
# --------------------------------------------------------
# Cargar modelo YOLOv8
# --------------------------------------------------------
model = YOLO("yolov8n.pt")  # Usamos el modelo más ligero

# --------------------------------------------------------
# Función para detección de objetos con YOLOv8
# --------------------------------------------------------
def detect_objects(image_path):
    try:
        results = model(image_path)
        detections = results[0].boxes.data.cpu().numpy()
        
        if detections.size == 0:
            return pd.DataFrame(columns=['name', 'confidence', 'image'])

        df_results = pd.DataFrame(detections, columns=['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class'])
        df_results['name'] = df_results['class'].apply(lambda x: model.names[int(x)])
        df_results['image'] = os.path.basename(image_path)
        return df_results[['name', 'confidence', 'image']]

    except Exception as e:
        print(f"Error en {image_path}: {e}")
        return pd.DataFrame(columns=['name', 'confidence', 'image'])

# --------------------------------------------------------
# Función para procesar imágenes de manera secuencial
# --------------------------------------------------------
def procesar_imagenes(image_files):
    resultados_objetos = []
    
    # Procesar las imágenes una por una
    for image_path in tqdm(image_files, desc="Procesando imágenes"):
        # Realizar detección de objetos
        df_obj = detect_objects(image_path)
        
        if not df_obj.empty:
            resultados_objetos.append(df_obj)
    
    # Combinar los resultados de detección
    if resultados_objetos:
        df_objetos = pd.concat(resultados_objetos, ignore_index=True)
        return df_objetos
    else:
        return pd.DataFrame(columns=['name', 'confidence', 'image'])


# Porcentaje Verde

In [ ]:
# --------------------------------------------------------
# Función para calcular porcentaje de verde en la imagen
# --------------------------------------------------------
def porcentaje_verde(image_path):
    try:
        # Leemos la imagen
        img = cv2.imread(image_path)
        
        # Verificamos si la imagen es válida
        if img is None:
            print(f"Error: No se pudo leer la imagen {image_path}")
            return np.nan
        
        # Reducimos la resolución para reducir uso de memoria
        img = cv2.resize(img, (512, 512))
        
        # Convertimos la imagen a formato HSV
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        
        # Definimos el rango de verde en el espacio HSV
        lower = np.array([35, 40, 40])  # Rango inferior para verde
        upper = np.array([85, 255, 255])  # Rango superior para verde
        
        # Creamos una máscara con los valores dentro del rango de verde
        mask = cv2.inRange(hsv, lower, upper)
        
        # Calculamos el porcentaje de píxeles verdes
        verde_percentage = np.sum(mask > 0) / mask.size
        return verde_percentage
    
    except Exception as e:
        print(f"Error en porcentaje_verde para {image_path}: {e}")
        return np.nan

# --------------------------------------------------------
# Función para procesar las imágenes una por una
# --------------------------------------------------------
def procesar_porcentaje_verde(image_files):
    resultados_verde = []
    
    for image_path in tqdm(image_files, desc="Procesando porcentaje de verde"):
        verde = porcentaje_verde(image_path)
        resultados_verde.append(verde)
    
    # Extraer solo el nombre del archivo y crear DataFrame
    image_names = [os.path.basename(img) for img in image_files]
    # Crear DataFrame para los resultados
    df_verde = pd.DataFrame({'image': image_names, 'porcentaje_verde': resultados_verde})
    return df_verde

# Textura

In [ ]:
# --------------------------------------------------------
# FUNCIÓN: Calcula nivel de textura con gradiente (liviana)
# --------------------------------------------------------
def nivel_textura_gradiente(image_path):
    try:
        # Cargar en escala de grises
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"No se pudo leer: {image_path}")
            return np.nan

        # Redimensionar para ahorrar memoria
        img = cv2.resize(img, (256, 256))

        # Gradientes en X y Y con Sobel
        grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
        grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)

        # Magnitud del gradiente
        grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)

        # Desvío estándar como medida de textura
        texture_score = np.std(grad_magnitude)
        return texture_score

    except Exception as e:
        print(f"Error en {image_path}: {e}")
        return np.nan

# --------------------------------------------------------
# FUNCIÓN: Procesa imágenes por lotes
# --------------------------------------------------------
def procesar_textura_batch(image_files, batch_size=50):
    resultados_textura = []
    for i in tqdm(range(0, len(image_files), batch_size), desc="Calculando textura"):
        batch = image_files[i:i+batch_size]
        for img_path in batch:
            textura = nivel_textura_gradiente(img_path)
            resultados_textura.append(textura)
        gc.collect()
    # Extraer solo el nombre del archivo y crear DataFrame
    image_names = [os.path.basename(img) for img in image_files]
    return pd.DataFrame({'image': image_names, 'nivel_textura': resultados_textura})

# Embeddings

In [ ]:

# --------------------------------------------------------
# Cargar modelo preentrenado (ResNet18 sin la capa final)
# --------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = models.resnet18(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Quitar capa de clasificación
resnet.eval().to(device)

# --------------------------------------------------------
# Transformación de imágenes para ResNet
# --------------------------------------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Tamaño requerido por ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --------------------------------------------------------
# Función para extraer embedding visual
# --------------------------------------------------------
def extraer_embedding(image_path):
    try:
        image = Image.open(image_path).convert('RGB')
        image_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            embedding = resnet(image_tensor)
            embedding = embedding.squeeze().cpu().numpy()  # shape: (512,)
        return embedding
    except Exception as e:
        print(f"Error en {image_path}: {e}")
        return [float('nan')] * 512

# --------------------------------------------------------
# Función para procesar imágenes por lotes
# --------------------------------------------------------
def procesar_embeddings_batch(image_files, batch_size=50):
    embeddings = []
    nombres = []

    for i in tqdm(range(0, len(image_files), batch_size), desc="Extrayendo embeddings"):
        batch = image_files[i:i+batch_size]
        for img_path in batch:
            emb = extraer_embedding(img_path)
            embeddings.append(emb)
            nombres.append(img_path)
        gc.collect()
    # Extraer solo el nombre del archivo y crear DataFrame
    image_names = [os.path.basename(img) for img in nombres]
    df = pd.DataFrame(embeddings)
    df.insert(0, "image", image_names)
    return df
